In [1]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, item_knn as knn, basic
from lenskit import topn

In [2]:
import numpy as np
import pandas as pd
%matplotlib inline

In [3]:
ratings = pd.read_csv('/Users/denisehansen/Desktop/ITU/Thesis/Fair-Recommendations/ml-latest-small/ratings.1.csv', sep=',',
                      names=['user', 'item', 'rating', 'timestamp'], header=0)
ratings.head()

,user,item,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
algo_ii = knn.ItemItem(20)
algo_als = als.BiasedMF(50)

In [5]:
def eval(aname, algo, train, test):
    fittable = util.clone(algo)
    fittable = Recommender.adapt(fittable)
    fittable.fit(train)
    users = test.user.unique()
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [6]:
all_recs = []
test_data = []
for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 2, xf.SampleFrac(0.2)):
    test_data.append(test)
    all_recs.append(eval('ItemItem', algo_ii, train, test))
    all_recs.append(eval('ALS', algo_als, train, test))

test _reccomend
test _reccomend
test _reccomend
test _reccomend


In [7]:
all_recs = pd.concat(all_recs, ignore_index=True)
all_recs.head()

,item,score,user,rank,Algorithm
0,349,4.351950,3,1,ItemItem
1,480,4.351950,3,2,ItemItem
2,919,4.271184,3,3,ItemItem
3,1022,4.271184,3,4,ItemItem
4,141,4.271184,3,5,ItemItem


In [8]:
test_data = pd.concat(test_data, ignore_index=True)

In [9]:
rla = topn.RecListAnalysis()
rla.add_metric(topn.ndcg)
results = rla.compute(all_recs, test_data)
results.head()

Recs:
      item     score  user  rank Algorithm
392   2023  3.632020     1     1       ALS
393    266  3.496140     1     2       ALS
394     17  3.496140     1     3       ALS
395    585  3.496140     1     4       ALS
396    551  3.496140     1     5       ALS
397    265  3.496140     1     6       ALS
398    222  3.496140     1     7       ALS
399    590  3.473185     1     8       ALS
400    150  3.450231     1     9       ALS
401     39  3.450231     1    10       ALS
402   1221  3.370251     1    11       ALS
403   6711  3.370251     1    12       ALS
404    858  3.361686     1    13       ALS
405   2770  3.356080     1    14       ALS
406    435  3.317027     1    15       ALS
407   2903  3.317027     1    16       ALS
408   1805  3.317027     1    17       ALS
409   2616  3.317027     1    18       ALS
410    592  3.312390     1    19       ALS
411   1544  3.245730     1    20       ALS
412   4006  3.243453     1    21       ALS
413   1918  3.243453     1    22       ALS
414  

/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1472: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


ndcg
user Algorithm          
1    ALS        0.000000
     ItemItem   0.000000
2    ALS        0.034877
     ItemItem   0.166711
3    ALS        0.037721